# box-authentication
> Using box authentication

If you need to use Box authentication for accessing your files, use the document below to get started.  You'll need to do some setup steps on your own, so make sure that you read this document and refer to the associated reference files.

# Setting up Box for Apps
## Set up Box Developer
In order to access your files on Box, you'll need to set up an app.  Visit https://developer.box.com/ and click the `MyApps` button on the upper right hand corner. Sign in using your Vanderbilt email.  You'll be taken to single sign on where you'll enter your credentials.

Click the `Create New App` button.  Choose `Custom App` and then click the `User Authentication (Oauth 2.0)` radio button.  These two selections will enable you to be able to download and access your files without requiring explicit approval from the Enterprise (i.e., Vanderbilt) Box administrator.  More information on types of Box apps can be found here: https://developer.box.com/guides/applications/

Give your app a name.

## Configure your app access
Now that you've created your app, you'll set up the oauth parameters.  In the `redirect uri` box, type `http://localhost`.  Make sure the `Write all files and folders in Box` checkbox is checked since you'll want to download your files.  Click the blue `Save Changes` button.  Keep this page open because you'll need the `Client ID` and `Client Secret` fields for the next step.

## Create authentication file
In the main directory of this repo, create a file called `client_oauth.py`.  This will contain all of your access credentials.  **DO NOT** check this file into the repo.  In this file, you should have the following 3 lines, where `client_id` and `client_secret` are given by the values for your app.  Copy and paste them as a string here.  An example is shown below with fabricated values for client_id and client_secret:
```
client_id = 'sleirher87837483y42ksj'
client_secret = '34878rudhfu3437g73'
client_redirect_url = 'http://localhost'
```
Note that the redirect URI here should match the one set in your app on Box Developer.

# Using authentication within your project
Below are some functions that you can use within your project to first authenticate and then retrieve a file.  For more ways to interact with Box (e.g., accessing whole folders, listing folder contents, etc., see https://developer.box.com/guides/ ).

## Authentication dance
Below, you'll see two functions.  The first - `start_box_authentication` requires the imports of your created `client_oauth.py` file and the `Client` and `OAuth2` modules of the the `boxsdk` package.  See below for usage.  

After you've run this function, a new window will open which require you to sign into Box and allow access to the folders.  After you agree, you'll be redirected to localhost (which should fail to load page content).  Although there will be no page content, look at the url.  Near the middle of the url, you should see part of the url which starts with `code=lerwlerh2o3hyweiuhrlsiudjfn`.  Copy everything after the equals sign, and use that as the `access_token` input to the `read_data_from_box` function provided below.  You'll need to be quick - the code only lasts 10 seconds!

Make sure you have the `file_id` parameter set for the desired file (obtained from opening the file of interest on Box, and the `file_id` is the numerical value in the URL of the file/folder of interest).  Run this cell and your file should be read.

In [ ]:
#default_exp load_data
#all_no_test

In [ ]:
#export
import client_oauth
from boxsdk import Client, OAuth2
import pandas as pd

In [ ]:
#export
def start_box_authentication():
    '''
    Function start_box_authentication: starts box authentication and opens authorization window for Box access.  Make sure you copy
        the access code from the redirect window and input it into the read_data_from_box function within 10 seconds.
        Inputs: None
        Returns: OAuth2 instance to be used for authentication
    '''
    
    # create oauth
    oauth = OAuth2(
        client_id = client_oauth.client_id,
        client_secret = client_oauth.client_secret
    )
    
    #create authorization url
    auth_url, csrf_token = oauth.get_authorization_url(client_oauth.client_redirect_url)
    
    #redirect
    webbrowser.open(auth_url)
    
    return oauth

In [ ]:
# export
def read_data_from_box(oauth=None, access_token = None, file_id = None):
    '''
    Function read_data_from_box: reads file_id from Box using oauth provided by start_box_authentication and copied access token from redirect
        Inputs: oauth (default None): Oauth object e.g., output from start_box_authentication; no need to pass in if you have a persistent access token provided
                                      in your client_oauth.py file
                access_token (default None): String of code copied from the redirect url of start_box_authentication or persistent token stored in client_oauth.py file
                file_id (default None): String of Box file id to be read
        Returns: read file content
    '''
    
    # get authorization
    if oauth is None:
        oauth = OAuth2(
            client_id = client_oauth.client_id,
            client_secret = client_oauth.client_secret,
            access_token = client_oauth.access_token)
    else:
        oauth.authenticate(access_token)

    client = Client(oauth)

    # create file
    file_content = client.file(file_id).content()
    
    return file_content

# Example of usage

In [ ]:
# get authentication (imports and functions above must have been run)
oauth = start_box_authentication()

In [ ]:
# read data from file_id using copied access token
data = read_data_from_box(oauth, file_id = '87362564894736', access_token = 'luaefhlaiuehaliueig97ryfc')